In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from importlib import reload
import logging
reload(logging)
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%H:%M:%S')

In [3]:
import sys

sys.path.insert(0, '../..')

## Load Dataset

In [4]:
from entity_embed.benchmarks import DBLPScholarStructuredBenchmark

benchmark = DBLPScholarStructuredBenchmark(data_dir_path="../data/")
benchmark

17:21:59 INFO:Extracting DBLP-Scholar-Structured...
17:21:59 INFO:Reading DBLP-Scholar-Structured row_dict...
17:21:59 INFO:Reading DBLP-Scholar-Structured train.csv...
17:22:00 INFO:Reading DBLP-Scholar-Structured valid.csv...
17:22:00 INFO:Reading DBLP-Scholar-Structured test.csv...


<DBLPScholarStructuredBenchmark> from http://pages.cs.wisc.edu/~anhai/data1/deepmatcher_data/Structured/DBLP-GoogleScholar/dblp_scholar_exp_data.zip

## Preprocess

In [5]:
attr_list = ['title', 'authors', 'venue', 'year']

In [6]:
from tqdm.auto import tqdm
import unidecode

def clean_str(s):
    return unidecode.unidecode(s).lower().strip()

for row in tqdm(benchmark.row_dict.values()):
    for attr in attr_list:
        row[attr] = clean_str(row[attr])

  0%|          | 0/66879 [00:00<?, ?it/s]

## Init Data Module

In [7]:
import torch
import numpy as np

random_seed = 42
torch.manual_seed(random_seed)
np.random.seed(random_seed)

In [8]:
alphabet = list('0123456789abcdefghijklmnopqrstuvwxyz!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ ')

In [9]:
attr_config_dict = {
    'title': {
        'field_type': "MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    },
    'semantic_title': {
        'source_attr': 'title',
        'field_type': "SEMANTIC_MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'vocab': "fasttext.en.300d",
    },
    'authors': {
        'field_type': "MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    },
    'venue': {
        'field_type': "MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    },
    'semantic_venue': {
        'source_attr': 'venue',
        'field_type': "SEMANTIC_MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'vocab': "fasttext.en.300d",
    },
    'year': {
        'field_type': "STRING",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    }
}

In [10]:
from entity_embed import AttrConfigDictParser

row_numericalizer = AttrConfigDictParser.from_dict(attr_config_dict, row_list=benchmark.row_dict.values())
row_numericalizer.attr_config_dict

17:22:00 INFO:For attr=title, computing actual max_str_len
17:22:00 INFO:For attr=title, using actual_max_str_len=70
17:22:01 INFO:Loading vectors from .vector_cache/wiki.en.vec.pt
17:22:04 INFO:For attr=authors, computing actual max_str_len
17:22:05 INFO:actual_max_str_len=33 must be even to enable NN pooling. Updating to 34
17:22:05 INFO:For attr=authors, using actual_max_str_len=34
17:22:05 INFO:For attr=venue, computing actual max_str_len
17:22:05 INFO:actual_max_str_len=45 must be even to enable NN pooling. Updating to 46
17:22:05 INFO:For attr=venue, using actual_max_str_len=46
17:22:05 INFO:Loading vectors from .vector_cache/wiki.en.vec.pt
17:22:09 INFO:For attr=year, computing actual max_str_len
17:22:09 INFO:For attr=year, using actual_max_str_len=6


{'title': AttrConfig(source_attr='title', field_type=<FieldType.MULTITOKEN: 'multitoken'>, tokenizer='entity_embed.data_utils.numericalizer.default_tokenizer', alphabet=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~', ' '], max_str_len=70, vocab=None, n_channels=8, embed_dropout_p=0.2, use_attention=True, use_mask=True),
 'semantic_title': AttrConfig(source_attr='title', field_type=<FieldType.SEMANTIC_MULTITOKEN: 'semantic_multitoken'>, tokenizer='entity_embed.data_utils.numericalizer.default_tokenizer', alphabet=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '!', '"

In [11]:
datamodule = benchmark.build_datamodule(
    row_numericalizer=row_numericalizer,
    batch_size=32,
    eval_batch_size=128,
    random_seed=random_seed
)

## Training

In [12]:
from entity_embed import Matcher

model = Matcher(
    row_numericalizer=row_numericalizer,
)

In [13]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

max_epochs = 100
early_stop_callback = EarlyStoppingMinEpochs(
   monitor='valid_f1_at_0.5',
   min_delta=0.00,
   patience=20,
   verbose=True,
   mode='max'
)
tb_save_dir = '../tb_logs'
tb_name = f'matcher-{benchmark.dataset_name}'
trainer = pl.Trainer(
    gpus=1,
    max_epochs=max_epochs,
    check_val_every_n_epoch=1,
    callbacks=[early_stop_callback],
    logger=TensorBoardLogger(tb_save_dir, name=tb_name),
    reload_dataloaders_every_epoch=True
)

GPU available: True, used: True
17:22:09 INFO:GPU available: True, used: True
TPU available: None, using: 0 TPU cores
17:22:09 INFO:TPU available: None, using: 0 TPU cores


In [14]:
trainer.fit(model, datamodule)

17:22:09 INFO:Train pair count: 3207
17:22:09 INFO:Valid pair count: 1070
17:22:09 INFO:Test pair count: 1070
17:22:09 INFO:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name        | Type       | Params
-------------------------------------------
0 | blocker_net | BlockerNet | 31.1 M
1 | losser      | SupConLoss | 0     
-------------------------------------------
15.0 M    Trainable params
16.2 M    Non-trainable params
31.1 M    Total params
124.421   Total estimated model params size (MB)
17:24:50 INFO:
  | Name        | Type       | Params
-------------------------------------------
0 | blocker_net | BlockerNet | 31.1 M
1 | losser      | SupConLoss | 0     
-------------------------------------------
15.0 M    Trainable params
16.2 M    Non-trainable params
31.1 M    Total params
124.421   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

1

In [15]:
model.blocker_net.get_signature_weights()

{'title': 0.4005078971385956,
 'semantic_title': 0.31445467472076416,
 'authors': 0.16774962842464447,
 'venue': 0.06750933080911636,
 'semantic_venue': 0.019595319405198097,
 'year': 0.03018317185342312}

In [16]:
from entity_embed import validate_best

validate_best(trainer)

17:36:38 INFO:Validating best model...


{'valid_f1_at_0.3': 0.1386344442278958,
 'valid_f1_at_0.5': 0.6614761756462161,
 'valid_f1_at_0.7': 0.9307726663689146,
 'valid_f1_at_0.9': 0.9145383104125736,
 'valid_pair_entity_ratio_at_0.3': 7.518635170603675,
 'valid_pair_entity_ratio_at_0.5': 1.1238845144356955,
 'valid_pair_entity_ratio_at_0.7': 0.6136482939632546,
 'valid_pair_entity_ratio_at_0.9': 0.5070866141732283,
 'valid_precision_at_0.3': 0.07449556657124903,
 'valid_precision_at_0.5': 0.4960298925735638,
 'valid_precision_at_0.7': 0.8913601368691189,
 'valid_precision_at_0.9': 0.9637681159420289,
 'valid_recall_at_0.3': 0.997196261682243,
 'valid_recall_at_0.5': 0.9925233644859813,
 'valid_recall_at_0.7': 0.9738317757009346,
 'valid_recall_at_0.9': 0.8700934579439252}

## Testing

In [17]:
trainer.test(ckpt_path='best', verbose=False)

17:36:53 INFO:Train pair count: 3207
17:36:53 INFO:Valid pair count: 1070
17:36:53 INFO:Test pair count: 1070
17:36:53 INFO:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

[{'test_f1_at_0.3': 0.13798889892861751,
  'test_f1_at_0.5': 0.6645748352682774,
  'test_f1_at_0.7': 0.9400630914826499,
  'test_f1_at_0.9': 0.9125437281359321,
  'test_pair_entity_ratio_at_0.3': 7.5557883708748035,
  'test_pair_entity_ratio_at_0.5': 1.1089575694080671,
  'test_pair_entity_ratio_at_0.7': 0.6018858040859089,
  'test_pair_entity_ratio_at_0.9': 0.48768988999476165,
  'test_precision_at_0.3': 0.07411259012756517,
  'test_precision_at_0.5': 0.500236183278224,
  'test_precision_at_0.7': 0.9077458659704091,
  'test_precision_at_0.9': 0.9806659505907627,
  'test_recall_at_0.3': 0.9990654205607477,
  'test_recall_at_0.5': 0.9897196261682243,
  'test_recall_at_0.7': 0.974766355140187,
  'test_recall_at_0.9': 0.8532710280373832}]